In [3]:
import time
import logging
from pathlib import Path
import numpy as np
import astropy.units as u
from astropy.table import Table
from astropy.coordinates import Distance
from astropy.coordinates import SkyCoord
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier
from astroquery.xmatch import XMatch
from astroquery.heasarc import Heasarc
from utils import (
    get_source_identifier,
    insert_space_source_ids,
    convert_F_nu_to_luminosity,
)


log = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s|%(name)s|%(message)s",
    datefmt="%m/%d/%Y %I:%M:%S %p",
)

In [1]:
def find_xmatch_id_in_catalog(
    xmatch_id, xmatch_id_column, catalog, catalog_id_column, id_strip=None
):
    """Given a catalogue with cross-matched sources, return the sources
    corresponding to a given counterpart."""
    if id_strip is None:
        this_source = catalog[xmatch_id_column] == xmatch_id
    else:
        if catalog[xmatch_id_column].dtype == np.int64:
            this_source = catalog[xmatch_id_column] == np.int64(
                xmatch_id.strip(id_strip)
            )
        else:
            this_source = catalog[xmatch_id_column] == xmatch_id.strip(id_strip)

    xmatches = catalog[this_source]

    
    if len(xmatches) == 0:
        log.info(f"{xmatch_id} not matched in the catalogue")
    else:
        log.info(f"{xmatch_id} matched with {xmatches[catalog_id_column].data}")
        return xmatches

def convert_flux_to_luminosity(flux, flux_unit, distance):
    
    flux = flux * u.Unit(flux_unit)
    D_L = distance.to(u.cm) # Convert Mpc to cm for consistent units
    flux_in_cgs = flux.to(u.erg / (u.s * u.cm**2))
    luminosity = 4 * np.pi * D_L**2 * flux_in_cgs
    
    return luminosity.to(u.erg / u.s)

# Load the CoreG and FR0 catalogues

In [3]:
#CoreG Catalogs
ho_1997 = Vizier(columns=["**"], row_limit=-1).get_catalogs("J/ApJS/112/315")
nagar_2005 = Vizier(columns=["**"], row_limit=-1).get_catalogs("J/A+A/435/521")
#FR0 Catalog
fr0cat = Vizier(columns=["**"], row_limit=-1).get_catalogs("J/A+A/609/A1")
#MORX Catalog
morx = Vizier(columns=["**"], row_limit=-1).get_catalogs("V/158/morxv2")
torresi_sources = [
    "SDSS J004150.47−091811.2",
    "SDSS J010101.12−002444.4",
    "SDSS J011515.78+001248.4",
    "SDSS J015127.10−083019.3",
    "SDSS J080624.94+172503.7",
    "SDSS J092405.30+141021.5",
    "SDSS J093346.08+100909.0",
    "SDSS J094319.15+361452.1",
    "SDSS J104028.37+091057.1",
    "SDSS J114232.84+262919.9",
    "SDSS J115954.66+302726.9",
    "SDSS J122206.54+134455.9",
    "SDSS J125431.43+262040.6",
    "Tol 1326−379",
    "SDSS J135908.74+280121.3",
    "SDSS J153901.66+353046.0",
    "SDSS J160426.51+174431.1",
    "SDSS J171522.97+572440.2",
    "SDSS J235744.10−001029.9",
]

09/19/2025 01:51:44 PM|astroquery|UnitsWarning: Unit 'pct' not supported by the VOUnit standard. Did you mean PC, PT, Pct, pC, pT or pc?


In [4]:
# Replace 'your_file.csv' with the
#path to your CSV file
swift = Table.read('LSXPS_Sources.csv', format='csv')

In [6]:
flux = swift['FixedAPECUnabsFlux_neg'].quantity

In [13]:
swift[0]

LSXPS_ID,IAUName,RA,Decl,Err90,AstromType,l,b,MeanOffAxisAngle,NearestNeighbour,NearestOKNeighbour,NearestNeighbour_ID,NearestOKNeighbour_ID,HPPIX,Exposure,FirstObsDate,LastObsDate,FirstObsMET,LastObsMET,FirstDetDate,LastDetDate,FirstDetMET,LastDetMET,FirstBlindDetDate,LastBlindDetDate,FirstBlindDetMET,LastBlindDetMET,NumObs,NumBlindDetObs,NumDetObs,BestDetectionID,NonBlindDet_band0,NonBlindDet_band1,NonBlindDet_band2,NonBlindDet_band3,DetFlag,FieldFlag,DetFlag_band0,DetFlag_band1,DetFlag_band2,DetFlag_band3,OpticalLoadingWarning,StrayLightWarning,NearBrightSourceWarning,IsPotentialAlias,WhenAdded,StillDetected,ProcessedStatus,WhenModified,Rate_band0,Rate_band0_pos,Rate_band0_neg,HR1,HR1_pos,HR1_neg,HR2,HR2_pos,HR2_neg,Rate_band1,Rate_band1_pos,Rate_band1_neg,Rate_band2,Rate_band2_pos,Rate_band2_neg,Rate_band3,Rate_band3_pos,Rate_band3_neg,Counts_band0,Counts_band1,Counts_band2,Counts_band3,BgCounts_band0,BgCounts_band1,BgCounts_band2,BgCounts_band3,RateCF_band0,RateCF_band1,RateCF_band2,RateCF_band3,UL_band0,UL_band1,UL_band2,UL_band3,PeakRate_band0,PeakRate_band0_pos,PeakRate_band0_neg,PeakRate_band1,PeakRate_band1_pos,PeakRate_band1_neg,PeakRate_band2,PeakRate_band2_pos,PeakRate_band2_neg,PeakRate_band3,PeakRate_band3_pos,PeakRate_band3_neg,PvarPchiSnapshot_band0,PvarPchiSnapshot_band1,PvarPchiSnapshot_band2,PvarPchiSnapshot_band3,PvarPchiSnapshot_HR1,PvarPchiSnapshot_HR2,PvarPchiObsID_band0,PvarPchiObsID_band1,PvarPchiObsID_band2,PvarPchiObsID_band3,PvarPchiObsID_HR1,PvarPchiObsID_HR2,GalacticNH,WhichPow,WhichAPEC,PowECFO,PowECFU,PowFlux,PowFlux_pos,PowFlux_neg,PowUnabsFlux,PowUnabsFlux_pos,PowUnabsFlux_neg,APECECFO,APECECFU,APECFlux,APECFlux_pos,APECFlux_neg,APECUnabsFlux,APECUnabsFlux_pos,APECUnabsFlux_neg,PowPeakFlux,PowPeakFlux_pos,PowPeakFlux_neg,PowPeakUnabsFlux,PowPeakUnabsFlux_pos,PowPeakUnabsFlux_neg,APECPeakFlux,APECPeakFlux_pos,APECPeakFlux_neg,APECPeakUnabsFlux,APECPeakUnabsFlux_pos,APECPeakUnabsFlux_neg,FixedPowECFO,FixedPowECFU,FixedPowFlux,FixedPowFlux_pos,FixedPowFlux_neg,FixedPowUnabsFlux,FixedPowUnabsFlux_pos,FixedPowUnabsFlux_neg,FixedAPECECFO,FixedAPECECFU,FixedAPECFlux,FixedAPECFlux_pos,FixedAPECFlux_neg,FixedAPECUnabsFlux,FixedAPECUnabsFlux_pos,FixedAPECUnabsFlux_neg,InterpPowECFO,InterpPowECFU,InterpPowNH,InterpPowNH_pos,InterpPowNH_neg,InterpPowGamma,InterpPowGamma_pos,InterpPowGamma_neg,InterpPowFlux,InterpPowFlux_pos,InterpPowFlux_neg,InterpPowUnabsFlux,InterpPowUnabsFlux_pos,InterpPowUnabsFlux_neg,InterpAPECECFO,InterpAPECECFU,InterpAPECNH,InterpAPECNH_pos,InterpAPECNH_neg,InterpAPECkT,InterpAPECkT_pos,InterpAPECkT_neg,InterpAPECFlux,InterpAPECFlux_pos,InterpAPECFlux_neg,InterpAPECUnabsFlux,InterpAPECUnabsFlux_pos,InterpAPECUnabsFlux_neg,P_pow,P_APEC,FittedPowECFO,FittedPowECFU,FittedPowNH,FittedPowNH_pos,FittedPowNH_neg,FittedPowGamma,FittedPowGamma_pos,FittedPowGamma_neg,FittedPowFlux,FittedPowFlux_pos,FittedPowFlux_neg,FittedPowUnabsFlux,FittedPowUnabsFlux_pos,FittedPowUnabsFlux_neg,FittedPowCstat,FittedPowDOF,FittedPowReducedChi2,FittedAPECECFO,FittedAPECECFU,FittedAPECNH,FittedAPECNH_pos,FittedAPECNH_neg,FittedAPECkT,FittedAPECkT_pos,FittedAPECkT_neg,FittedAPECFlux,FittedAPECFlux_pos,FittedAPECFlux_neg,FittedAPECUnabsFlux,FittedAPECUnabsFlux_pos,FittedAPECUnabsFlux_neg,FittedAPECCstat,FittedAPECDOF,FittedAPECReducedChi2,HasSpec,NumExternalMatches,NumExternalMatches_slim,MatchInROSHRI,MatchIn2RXS,MatchIn4XMM_DR10,MatchIn4XMM_DR10s,MatchInXMMSL2,MatchInSwiftFT,MatchIn1SWXRT,MatchInXRTGRB,MatchInSDSS_QSO_DR14,MatchIn2MASS,MatchInUSNOB1,MatchIn2CSC,MatchIn2SXPS
int64,str22,float64,float64,float64,int64,float64,float64,float64,float64,float64,int64,int64,int64,int64,str19,str19,int64,int64,str19,str19,int64,int64,str19,str19,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,int64,int64,int64,str19,int64,int64,str19,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,fl

In [6]:
#4XMM Catalog
xmm4 = Vizier(columns=["**"], row_limit=-1).get_catalogs("IX/69/xmm4d13s")
cxo = Vizier(columns=["**"], row_limit=-1).get_catalogs("IX/45/csc11")

In [8]:
coreG_catalogue = Table(
    names=(
        "SOURCE_NAME",
        "SOURCE_TYPE",
        "SIMBAD SDSS-ID",
        "SIMBAD FERMI-ID",
        "SIMBAD NVSS-ID",
        "NVSS-XMATCH-ID",
        "NVSS-MORX-ID",
        "SIMBAD FIRST-ID",
        "FIRST-XMATCH-ID",
        "FIRST-MORX-ID",
        "XMM-MORX-ID",
        "CXO-MORX-ID",
        "SWIFT-MORX-ID",
        "LoTSS-MORX-ID",
        "VLASS-MORX",
        "LOBE EXTENSION",
        "DISTANCE",
        "Log10(L_OIII)",
        "TORESSI DETECTION",
        "NVSS-FLUX-XMATCH",
        "NVSS-FLUX-ERROR-XMATCH",
        "FIRST-FLUX-XMATCH",
        "FIRST-FLUX-ERROR-XMATCH",
        "4XMM Mean Flux (2.0-4.5keV band)",
        "4XMM Mean Flux Error (2.0-4.5keV band)",
        "4XMM Mean Flux (4.5-12keV band)",
        "4XMM Mean Flux Error (4.5-12keV band)",
        "4XMM Mean Flux (0.2-12keV band)",
        "4XMM Mean Flux Error (0.2-12keV band)",
        "ACIS Flux F_b",
        "F_b lowlim",
        "F_b uplim",
        "HRC Flux F_w",
        "F_w lowlim",
        "F_w uplim",
        "Swift PowerLaw-Flux",
        "Swift Flux Neg",
        "Swift Flux Pos",
        "Hardness Ratio 4-5",
    ),
    dtype=[
        str,
        str,
        str,
        str,
        str,
        str,
        str,
        str,
        str,
        str,
        str,
        str,
        str,
        str,
        str,
        np.int16,
        np.float32,
        np.float32,
        bool,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
    ],
    units=[
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "mas",
        "Mpc",
        "erg s-1",
        "",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "",
    ],
)

fr0_catalogue = Table(
    names=(
        "SOURCE_NAME",
        "SOURCE_TYPE",
        "SIMBAD SDSS-ID",
        "SIMBAD FERMI-ID",
        "SIMBAD NVSS-ID",
        "NVSS-XMATCH-ID",
        "NVSS-MORX-ID",
        "SIMBAD FIRST-ID",
        "FIRST-XMATCH-ID",
        "FIRST-MORX-ID",
        "XMM-MORX-ID",
        "CXO-MORX-ID",
        "SWIFT-MORX-ID",
        "LoTSS-MORX-ID",
        "VLASS-MORX",
        "LOBE EXTENSION",
        "DISTANCE",
        "Log10(L_OIII)",
        "TORESSI DETECTION",
        "NVSS-FLUX-XMATCH",
        "NVSS-FLUX-ERROR-XMATCH",
        "FIRST-FLUX-XMATCH",
        "FIRST-FLUX-ERROR-XMATCH",
        "4XMM Mean Flux (2.0-4.5keV band)",
        "4XMM Mean Flux Error (2.0-4.5keV band)",
        "4XMM Mean Flux (4.5-12keV band)",
        "4XMM Mean Flux Error (4.5-12keV band)",
        "4XMM Mean Flux (0.2-12keV band)",
        "4XMM Mean Flux Error (0.2-12keV band)",
        "ACIS Flux F_b",
        "F_b lowlim",
        "F_b uplim",
        "HRC Flux F_w",
        "F_w lowlim",
        "F_w uplim",
        "Swift PowerLaw-Flux",
        "Swift Flux Neg",
        "Swift Flux Pos",
        "Hardness Ratio 4-5",
    ),
    dtype=[
        str,
        str,
        str,
        str,
        str,
        str,
        str,
        str,
        str,
        str,
        str,
        str,
        str,
        str,
        str,
        np.int16,
        np.float32,
        np.float32,
        bool,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
        np.float64,
    ],
    units=[
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "mas",
        "Mpc",
        "erg s-1",
        "",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "",
    ],
)



In [9]:
first_xmatches = XMatch.query(
    cat1=nagar_2005[0],
    cat2="vizier:VIII/92/first14",
    colRA1="_RA",
    colDec1="_DE",
    colRA2="RAJ2000",
    colDec2="DEJ2000",
    max_distance=10 * u.arcsec,
)

first_xmatches

angDist,recno,Name,AType,TType,Dist,RAObs,DEObs,Delta,Sp15GHz,l_St15GHz,St15GHz,l_Lp15GHz,Lp15GHz,Note,Simbad,NED,_RA,_DE,_RAJ2000,_DEJ2000,FIRST,FITS,RAJ2000,DEJ2000,p(S),Fpeak,Fint,Rms,Maj,Min,PA,N1,c1,N2,Ep,Ep(JD),s_Ep(JD)
arcsec,,,,,,hms,dms,,,,,,,,,,,,deg,deg,,,"""h:m:s""","""d:m:s""",,mJy,mJy,mJy,arcsec,arcsec,deg,,,,yr,d,d
float64,int16,str7,str9,float32,float32,str12,str12,float32,float32,str1,float32,str1,float32,int16,str6,str3,float64,float32,float64,float64,str16,str4,str12,str12,float32,float64,float64,float32,float32,float32,float32,int16,str1,int16,float64,float64,float64
0.944799,14,NGC524,T2:,-1.0,32.1,,,--,1.5,,1.5,,20.27,3,Simbad,NED,21.1992,9.5389,21.198963,9.538781,J012447.7+093219,FITS,01 24 47.751,+09 32 19.61,0.014,1.11,0.72,0.132,0.96,0.0,90.2,2,g,2,2009.378,2454970.4,0.348
5.977018,20,NGC1055,T2/L2::,3.0,12.6,,,--,--,<,1.8,<,19.53,0,Simbad,NED,40.4387,0.4423,40.438246,0.443897,J024145.1+002638,FITS,02 41 45.179,+00 26 38.03,0.014,6.91,11.9,0.096,5.91,3.81,94.5,0,,3,1998.142,2450866.2,1174.939
2.125878,30,NGC2681,L1.9,0.0,13.3,,,--,--,<,1.4,<,19.48,0,Simbad,NED,133.386,51.3132,133.386462,51.313717,J085332.7+511849,FITS,08 53 32.751,+51 18 49.38,0.014,3.78,4.68,0.144,3.31,1.83,35.4,1,g,1,1997.32,2450566.4,0.851
9.560043,31,NGC2683,L2/S2,3.0,5.7,,,--,--,<,0.9,<,18.54,0,Simbad,NED,133.1737,33.4196,133.17205,33.421872,J085241.2+332518,FITS,08 52 41.292,+33 25 18.74,0.023,1.19,2.09,0.133,5.35,4.03,23.0,2,g,5,1994.504,2449537.6,84.129
0.892253,33,NGC2768,L2,-5.0,23.7,09 11 37.413,+60 02 14.86,0.9,7.9,,8.2,,20.73,0,Simbad,NED,137.906,60.0372,137.905862,60.037439,J091137.4+600214,FITS,09 11 37.407,+60 02 14.78,0.014,11.7,12.26,0.147,1.5,0.77,19.0,0,,1,2002.51,2452461.7,0.962
7.141463,36,NGC2841,L2,3.0,12.0,09 22 02.678,+50 58 35.70,7.4,1.1,,2.1,,19.28,0,Simbad,NED,140.5085,50.9777,140.511087,50.976567,J092202.6+505835,FITS,09 22 02.661,+50 58 35.64,0.014,1.97,3.0,0.143,5.56,1.91,18.5,1,g,1,1997.314,2450564.2,1.182
0.331187,38,NGC2911,L2,-2.0,42.2,09 33 46.108,+10 09 08.82,0.5,17.3,,17.7,,21.57,0,Simbad,NED,143.4422,10.1524,143.442112,10.152431,J093346.1+100908,FITS,09 33 46.107,+10 09 08.75,0.014,42.0,45.59,0.151,1.96,1.1,104.1,1,g,1,2000.036,2451557.8,1.635
7.12145,41,NGC3079,S2,7.0,20.4,,,--,48.0,,59.0,,21.38,7,Simbad,NED,150.4939,55.6806,150.490704,55.679783,J100157.7+554047,FITS,10 01 57.769,+55 40 47.22,0.014,157.08,293.04,0.15,5.53,4.51,169.2,1,g,6,1997.377,2450587.0,0.861


In [10]:
nvss_xmatches = XMatch.query(
    cat1=nagar_2005[0],
    cat2="vizier:VIII/65/nvss",
    colRA1="_RA",
    colDec1="_DE",
    colRA2="RAJ2000",
    colDec2="DEJ2000",
    max_distance=10 * u.arcsec,
)

In [11]:
nvss_xmatches

angDist,recno,Name,AType,TType,Dist,RAObs,DEObs,Delta,Sp15GHz,l_St15GHz,St15GHz,l_Lp15GHz,Lp15GHz,Note,Simbad,NED,_RA,_DE,_RAJ2000,_DEJ2000,NVSS,RAJ2000,DEJ2000,e_RAJ2000,e_DEJ2000,S1.4,e_S1.4,l_MajAxis,MajAxis,l_MinAxis,MinAxis,f_resFlux
arcsec,,,,,,hms,dms,,,,,,,,,,,,deg,deg,,"""h:m:s""","""d:m:s""",s,arcsec,mJy,mJy,,arcsec,,arcsec,
float64,int16,str7,str9,float32,float32,str12,str12,float32,float32,str1,float32,str1,float32,int16,str6,str3,float64,float32,float64,float64,str14,str11,str11,float32,float32,float64,float32,str1,float32,str1,float32,str2
2.695816,6,NGC266,L1.9,2.0,62.4,00 49 47.819,+32 16 39.71,2.0,4.1,,4.1,,21.28,0,Simbad,NED,12.4499,32.2776,12.44942,32.27697,004947+321637,00 49 47.86,+32 16 37.1,0.29,4.0,8.0,1.1,,45.7,<,56.1,
9.129749,7,NGC315,L1.9,-4.0,65.8,,,--,470.0,,--,,23.39,9,Simbad,NED,14.4537,30.3524,14.45142,30.354,005748+302114,00 57 48.34,+30 21 14.4,0.03,0.6,772.1,25.3,,39.1,<,13.9,P*
6.759193,8,NGC404,L2,-3.0,2.4,,,--,--,<,1.3,<,17.95,0,Simbad,NED,17.3626,35.718,17.36475,35.71869,010927+354307,01 09 27.54,+35 43 07.3,0.35,5.1,3.4,0.4,<,57.0,<,44.2,
1.812751,9,NGC410,T2:,-4.0,70.6,,,--,--,<,1.0,<,20.78,4,Simbad,NED,17.746,33.152,17.74558,33.15164,011058+330905,01 10 58.94,+33 09 05.9,0.27,2.7,5.8,0.5,<,49.0,<,36.3,
2.505663,14,NGC524,T2:,-1.0,32.1,,,--,1.5,,1.5,,20.27,3,Simbad,NED,21.1992,9.5389,21.19867,9.53936,012447+093221,01 24 47.68,+09 32 21.7,0.4,5.3,3.1,0.4,<,61.5,<,49.0,
1.536308,18,NGC777,S2/L2::,-5.0,66.5,,,--,--,<,1.5,<,20.9,0,Simbad,NED,30.0625,31.4293,30.062,31.42931,020014+312545,02 00 14.88,+31 25 45.5,0.18,2.0,7.0,0.5,<,37.0,<,27.9,
6.800599,19,NGC841,L1.9:,2.1,59.5,,,--,--,<,1.5,<,20.8,0,Simbad,NED,32.8229,37.4969,32.82071,37.49764,021116+372951,02 11 16.97,+37 29 51.5,0.33,4.0,4.9,0.5,<,61.0,<,43.0,
7.250545,20,NGC1055,T2/L2::,3.0,12.6,,,--,--,<,1.8,<,19.53,0,Simbad,NED,40.4387,0.4423,40.43762,0.444,024145+002638,02 41 45.03,+00 26 38.4,0.04,0.6,198.5,6.7,,82.2,,28.5,S*


In [12]:
morx_nagar_xmatches = XMatch.query(
    cat1=nagar_2005[0],
    cat2="vizier:V/158/morxv2",
    colRA1="_RA",
    colDec1="_DE",
    colRA2="RAJ2000",
    colDec2="DEJ2000",
    max_distance=1 * u.arcsec,
)

In [13]:
morx_nagar_xmatches

angDist,recno,Name,AType,TType,Dist,RAObs,DEObs,Delta,Sp15GHz,l_St15GHz,St15GHz,l_Lp15GHz,Lp15GHz,Note,Simbad,NED,_RA,_DE,_RAJ2000,_DEJ2000,recno2,RAJ2000,DEJ2000,Name2,Type,Rmag,Bmag,Comment,R,B,z,Cite,Zcite,RXpct,QSOpct,galpct,starpct,Xraypct,CX-ID,XMM-ID,Swift-ID,OtherXID,radiopct,VLASS-ID,FIRST-ID,LoTSS-ID,RACS-ID,NVSS-ID,Lobe1,Lobe2,Lobedist
arcsec,,,,,,hms,dms,,,,,,,,,,,,deg,deg,,deg,deg,,,mag,mag,,,,,,,pct,pct,pct,pct,pct,,,,,pct,,,,,,,,arcsec
float64,int16,str7,str9,float32,float32,str12,str12,float32,float32,str1,float32,str1,float32,int16,str6,str3,float64,float32,float64,float64,int32,float64,float64,str25,str4,float32,float32,str3,str1,str1,float32,str6,str6,int16,int16,int16,int16,int16,str22,str22,str22,str22,int16,str22,str22,str22,str22,str22,str22,str22,int16
0.667458,6,NGC266,L1.9,2.0,62.4,00 49 47.819,+32 16 39.71,2.0,4.1,,4.1,,21.28,0,Simbad,NED,12.4499,32.2776,12.4500278,32.27775,114505,12.4500278,32.27775,NGC 266,GRX,7.82,10.19,p,1,1,0.015,0845,0845,95,2,89,4,90,CXOG J004947.8+321640,,LSXPSJ004947.7+321640,,91,VLAJ004947.80+321639.8,,ILTJ004947.82+321640.3,,NVSS J004947.8+321636,,,0
0.100097,7,NGC315,L1.9,-4.0,65.8,,,--,470.0,,--,,23.39,9,Simbad,NED,14.4537,30.3524,14.453673,30.352415,133914,14.453673,30.352415,NGC 315,GR2X,11.13,11.8,j+m,-,1,0.016,2214,2214,100,40,47,13,100,CXO J005748.4+302108,4XMM J005748.8+302109,LSXPSJ005748.9+302107,1RXH J005749.1+302110,99,VLAJ005748.74+302110.2,,ILDJ005748.85+302109.2,,,ILDJ005750.54+302051.0,ILDJ005746.15+302139.9,47
0.311808,8,NGC404,L2,-3.0,2.4,,,--,--,<,1.3,<,17.95,0,Simbad,NED,17.3626,35.718,17.3625084,35.7180429,160337,17.3625084,35.7180429,NGC 404,GRX,13.31,14.4,G,n,n,0.001,1592,0842,100,2,77,21,100,CXOG J010927.0+354305,,,1RXH J010926.6+354306,99,VLAJ010926.96+354304.9,,ILTJ010926.98+354305.2,,,,,0
0.136764,24,NGC1275,S1.5,10.0,70.1,,,--,2970.0,,--,,24.24,10,Simbad,NED,49.9507,41.5117,49.9506711,41.5117309,391017,49.9506711,41.5117309,NGC 1275,ARX,11.59,12.67,je,1,1,0.017,0096,0845,94,41,53,0,87,CXOG J031948.2+413041,4XMM J031948.2+413038,,,90,,,,,NVSS J031948.1+413041,,,0
0.757655,26,NGC2273,S2,0.5,28.4,06 50 08.652,+60 50 44.95,0.8,4.1,,5.1,,20.6,0,Simbad,NED,102.536,60.8456,102.5361353,60.8458,573549,102.5361353,60.8458,NGC 2273,ARX,8.22,9.99,p+G,1,1,0.006,PGC,1305,99,0,97,2,97,,4XMM J065008.6+605044,LSXPSJ065009.2+605041,1RXH J065008.5+605046,99,VLAJ065008.61+605044.7,,,,NVSS J065008.6+605044,,,0
0.03606,38,NGC2911,L2,-2.0,42.2,09 33 46.108,+10 09 08.82,0.5,17.3,,17.7,,21.57,0,Simbad,NED,143.4422,10.1524,143.4422,10.15239,969637,143.4422,10.15239,NGC 2911,NRX,15.55,16.31,g,1,1,0.011,PGC,2MAGN,98,25,72,1,91,,4XMM J093346.0+100909,,,97,VLAJ093346.09+100908.9,FIRSTJ093346.1+100908,,RACD J093346.0+100908,NVSS J093346.1+100909,,,0
0.220401,39,NGC2985,T1.9,2.0,22.4,,,--,--,<,1.0,<,19.78,4,Simbad,NED,147.5925,72.279,147.5924321,72.2789417,1032279,147.5924321,72.2789417,NGC 2985,ARX,10.66,12.29,pG,1,-,0.004,PGC,0845,100,3,85,12,99,CXOG J095022.1+721644,,,,100,VLAJ095022.17+721644.2,,,,NVSS J095022.6+721646,,,0
0.035088,40,NGC3031,S1.5,2.0,3.6,09 55 33.175,+69 03 55.06,6.4,164.1,,164.8,,20.41,0,Simbad,NED,148.8882,69.0653,148.8882236,69.0652951,1051645,148.8882236,69.0652951,NGC 3031,GRX,11.47,12.93,G,n,n,0.0,1592,2427,99,30,68,1,98,CXOG J095533.2+690355,4XMM J095533.2+690355,LSXPSJ095533.2+690355,1RXH J095533.6+690353,98,VLAJ095533.07+690354.5,,,,NVSS J095533.2+690354,,,0


In [14]:
nagar_2005[0]

recno,Name,AType,TType,Dist,RAObs,DEObs,Delta,Sp15GHz,l_St15GHz,St15GHz,l_Lp15GHz,Lp15GHz,Note,Simbad,NED,_RA,_DE
,,,,Mpc,,,arcsec,mJy,,mJy,,W / Hz,,,,deg,deg
int32,str7,str9,float32,float32,str12,str12,float32,float32,str1,float32,str1,float32,int16,str6,str3,float64,float64
1,IC239,L2::,6.0,16.8,,,--,--,<,0.9,<,19.48,--,Simbad,NED,39.1163,38.9690
2,IC356,T2,2.0,18.1,,,--,--,<,1.0,<,19.59,4,Simbad,NED,61.9436,69.8126
3,IC520,T2:,2.0,47.0,,,--,--,<,1.0,<,20.42,4,Simbad,NED,133.4258,73.4908
4,IC1727,T2/L2,9.0,8.2,,,--,--,<,0.9,<,18.86,--,Simbad,NED,26.8748,27.3330
5,NGC185,S2,-5.0,0.7,,,--,--,<,1.1,<,16.81,--,Simbad,NED,9.7392,48.3373
6,NGC266,L1.9,2.0,62.4,00 49 47.819,+32 16 39.71,2.0,4.1,,4.1,,21.28,--,Simbad,NED,12.4499,32.2776
7,NGC315,L1.9,-4.0,65.8,,,--,470.0,,--,,23.39,9,Simbad,NED,14.4537,30.3524
8,NGC404,L2,-3.0,2.4,,,--,--,<,1.3,<,17.95,--,Simbad,NED,17.3626,35.7180


In [15]:
for name, _type, distance, F_15GHz in zip(
    nagar_2005[0]["Name"],
    nagar_2005[0]["AType"],
    nagar_2005[0]["Dist"],
    nagar_2005[0]["St15GHz"],
):
    if "L" in _type:
        log.info(f"considering {name}")
        sdss_id = get_source_identifier(name,"SDSS")
        fermi_id = get_source_identifier(name, "4FGL")
        nvss_id = get_source_identifier(name, "NVSS")
        first_id = get_source_identifier(name, "FIRST")
        distancee =   distance * u.Mpc
        
        match_ho = ho_1997[1]["Name"] == insert_space_source_ids(name)
        if np.any(match_ho):  # Check if there is at least one True
            _log_L_alpha = ho_1997[1]["logL(Ha)"][match_ho][0]
            _OIII = ho_1997[1]["[OIII]"][match_ho][0]
            L_OIII = np.power(10, _log_L_alpha) * _OIII * u.Unit("erg s-1")
        else:
            L_OIII = 0 * u.Unit("erg s-1")
        
        # NVSS cross match with Nagar 2005 measurement
        this_source_nvss_xmatch = nvss_xmatches["Name"] == name
        if this_source_nvss_xmatch.any():
            nvss_xmatch_name = (
                "NVSS J" + nvss_xmatches["NVSS"][this_source_nvss_xmatch][0]
            )
            nvss_xmatch_flux = nvss_xmatches["S1.4"][this_source_nvss_xmatch][0]
            nvss_xmatch_flux_err = nvss_xmatches["e_S1.4"][this_source_nvss_xmatch][0]
        else:
            nvss_xmatch_name = ""
            nvss_xmatch_flux = 0
            nvss_xmatch_flux_err = 0

        # NVSS Cross matched with Nagar and then with MORX 
        nvss_morx_match = nvss_xmatch_name == morx_nagar_xmatches["NVSS-ID"]
        if nvss_morx_match.any():
            morx_nvss_name = morx_nagar_xmatches["NVSS-ID"][nvss_morx_match][0]
        else:
            morx_nvss_name = ""

        # FIRST measurement
        this_source_first_xmatch = first_xmatches["Name"] == name
        if this_source_first_xmatch.any():
            first_xmatch_name = (
                "FIRST " + first_xmatches["FIRST"][this_source_first_xmatch][0]
            )
            first_xmatch_name_nospace = (
                "FIRST" + first_xmatches["FIRST"][this_source_first_xmatch][0]
            )
            first_xmatch_flux = first_xmatches["Fint"][this_source_first_xmatch][0]
            first_xmatch_flux_err = first_xmatches["Rms"][this_source_first_xmatch][0]
            
        else:
            first_xmatch_name = ""
            first_xmatch_name_nospace = ""
            first_xmatch_flux = 0
            first_xmatch_flux_err = 0
            
        first_morx_match = first_xmatch_name_nospace == morx_nagar_xmatches["FIRST-ID"]
        if first_morx_match.any():
            morx_first_name = morx_nagar_xmatches["FIRST-ID"][first_morx_match][0]
        else:
            morx_first_name = ""

        morx_matches = morx_nagar_xmatches["Name"] == name
        if morx_matches.any():
            morx_xmm = morx_nagar_xmatches["XMM-ID"][morx_matches][0]
            xmm4_name = morx_xmm.removeprefix("4XMM ")
            morx_cxo = morx_nagar_xmatches["CX-ID"][morx_matches][0]
            cx0_name = morx_cxo.removeprefix("CXOG ")
            morx_swift = morx_nagar_xmatches["Swift-ID"][morx_matches][0]
            swift_name = insert_space_source_ids(morx_swift)
            morx_first = morx_nagar_xmatches["FIRST-ID"][morx_matches][0]
            morx_nvss = morx_nagar_xmatches["NVSS-ID"][morx_matches][0]
            morx_lotss = morx_nagar_xmatches["LoTSS-ID"][morx_matches][0]
            morx_vlass = morx_nagar_xmatches["VLASS-ID"][morx_matches][0]
            morx_lobedist = morx_nagar_xmatches["Lobedist"][morx_matches][0]
        else:
            morx_xmm = ""
            morx_cxo = ""
            xmm4_name = ""
            cx0_name = ""
            morx_swift = ""
            swift_name = ""
            morx_first = ""
            morx_nvss = ""
            morx_lotss = ""
            morx_vlass = ""
            morx_lobedist = 0
        
        morx_4xmm_match = xmm4[0]["4XMM"] == xmm4_name
        if morx_4xmm_match.any():
            xmm_flux4 = xmm4[0]["Flux4"][morx_4xmm_match][0]
            xmm_flux4_err = xmm4[0]["e_Flux4"][morx_4xmm_match][0]
            xmm_flux5 = xmm4[0]["Flux5"][morx_4xmm_match][0]
            xmm_flux5_err = xmm4[0]["e_Flux5"][morx_4xmm_match][0]
            xmm_flux9 = xmm4[0]["Flux9"][morx_4xmm_match][0]
            xmm_flux9_err = xmm4[0]["e_Flux9"][morx_4xmm_match][0]
            HR = xmm4[0]["HR4"][morx_4xmm_match][0]
        else:
            xmm_flux4 = 0
            xmm_flux4_err = 0
            xmm_flux5 = 0
            xmm_flux5_err = 0
            xmm_flux9 = 0
            xmm_flux9_err = 0
            HR = 0

        morx_cxo_match = cxo[0]["CXO"] == cx0_name
        if morx_cxo_match.any():
            cxo_fb = cxo[0]["Fb"][morx_cxo_match][0]
            cxo_fb_uplim = cxo[0]["B_Fb"][morx_cxo_match][0]
            cxo_fb_lowlim = cxo[0]["b_Fb"][morx_cxo_match][0]
            cxo_fw= cxo[0]["Fw"][morx_cxo_match][0]
            cxo_fw_uplim = cxo[0]["B_Fw"][morx_cxo_match][0]
            cxo_fw_lowlim = cxo[0]["b_Fw"][morx_cxo_match][0]
        else:
            cxo_fb = 0
            cxo_fb_uplim = 0
            cxo_fb_lowlim = 0
            cxo_fw = 0
            cxo_fw_uplim = 0
            cxo_fw_lowlim = 0            

        morx_swift_match = swift["IAUName"] == swift_name
        if morx_swift_match.any():
            swift_flux = swift["PowFlux"][morx_swift_match][0]
            swift_flux_pos = swift["PowFlux_pos"][morx_swift_match][0]
            swift_flux_neg = swift["PowFlux_neg"][morx_swift_match][0]
        else: 
            swift_flux = 0
            swift_flux_pos = 0
            swift_flux_neg = 0
        
        # check if the source is in the list of sources detected by Torresi et al. 2018
        torresi_detection = sdss_id in torresi_sources

        coreG_catalogue.add_row(
            [
                name,
                _type,
                sdss_id,
                fermi_id,
                nvss_id,
                nvss_xmatch_name,
                morx_nvss_name,
                first_id,
                first_xmatch_name,
                morx_first_name,
                morx_xmm,
                morx_cxo,
                morx_swift,
                morx_lotss,
                morx_vlass,
                morx_lobedist,
                distancee,
                np.log10(L_OIII.to_value("erg s-1")),
                torresi_detection,
                convert_F_nu_to_luminosity(1.4 * u.GHz, nvss_xmatch_flux, u.mJy, distancee),
                convert_F_nu_to_luminosity(1.4 * u.GHz, nvss_xmatch_flux_err, u.mJy, distancee),
                convert_F_nu_to_luminosity(1.4 * u.GHz, first_xmatch_flux, u.mJy, distancee),
                convert_F_nu_to_luminosity(1.4 * u.GHz, first_xmatch_flux_err, u.mJy, distancee),
                convert_flux_to_luminosity(xmm_flux4,'mW / m**2', distancee),
                convert_flux_to_luminosity(xmm_flux4_err,'mW / m**2', distancee),
                convert_flux_to_luminosity(xmm_flux5,'mW / m**2', distancee),
                convert_flux_to_luminosity(xmm_flux5_err,'mW / m**2', distancee),
                convert_flux_to_luminosity(xmm_flux9,'mW / m**2', distancee),
                convert_flux_to_luminosity(xmm_flux9_err,'mW / m**2', distancee), 
                convert_flux_to_luminosity(cxo_fb,'mW / m**2', distancee),
                convert_flux_to_luminosity(cxo_fb_lowlim,'mW / m**2', distancee),
                convert_flux_to_luminosity(cxo_fb_uplim,'mW / m**2', distancee),
                convert_flux_to_luminosity(cxo_fw,'mW / m**2', distancee),
                convert_flux_to_luminosity(cxo_fw_lowlim,'mW / m**2', distancee),
                convert_flux_to_luminosity(cxo_fw_uplim,'mW / m**2', distancee),
                convert_flux_to_luminosity(swift_flux,'erg / (cm**2)*(s)', distancee),
                convert_flux_to_luminosity(swift_flux_neg,'erg / (cm**2)*(s)', distancee),
                convert_flux_to_luminosity(swift_flux_pos,'erg / (cm**2)*(s)', distancee),
                HR
            ])

09/15/2025 12:14:19 PM|__main__|considering IC239
09/15/2025 12:14:19 PM|utils|SDSS counterpart not available for IC239
09/15/2025 12:14:19 PM|utils|4FGL counterpart not available for IC239
09/15/2025 12:14:19 PM|utils|NVSS counterpart not available for IC239
09/15/2025 12:14:19 PM|utils|FIRST counterpart not available for IC239
/opt/anaconda3/lib/python3.12/site-packages/astropy/table/table.py:3220: UserWarning: Units from inserted quantities will be ignored.
  self.insert_row(len(self), vals, mask)
09/15/2025 12:14:20 PM|__main__|considering IC1727
09/15/2025 12:14:20 PM|utils|SDSS counterpart not available for IC1727
09/15/2025 12:14:20 PM|utils|4FGL counterpart not available for IC1727
09/15/2025 12:14:20 PM|utils|NVSS counterpart not available for IC1727
09/15/2025 12:14:20 PM|utils|FIRST counterpart not available for IC1727
09/15/2025 12:14:20 PM|__main__|considering NGC266
09/15/2025 12:14:20 PM|utils|NGC266 matched with SDSS J004947.81+321639.8 by SIMBAD
09/15/2025 12:14:20 PM|

In [16]:
np.array(coreG_catalogue['SIMBAD FERMI-ID'])

array(['', '', '', '4FGL J0057.7+3023', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '4FGL J1219.6+0550', '', '', '', '', '', '', '', '', '',
       '4FGL J1230.8+1223', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', ''], dtype='<U17')

In [17]:
np.array(fr0_catalogue['SIMBAD FERMI-ID'])

array([], dtype='<U1')

In [18]:
np.array(coreG_catalogue["HRC Flux F_w"])

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

# FR0 Sources

In [19]:
fr0cat[0]

recno,SDSS,z,F1.4GHz,F[OIII],rmag,Dn,sigma*,Cr,logLr,logL[OIII],logMBH,SimbadName,_RA,_DE
,,,mJy,1e-20 W / m2,mag,,km / s,,log(1e-07W),log(1e-07W),log(solMass),,deg,deg
int32,str19,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,str24,float64,float64
1,J010852.48-003919.4,0.045,10.9,115.7,15.095,2.00,223,3.43,38.89,39.77,8.3,SDSS J010852.48-003919.4,17.21868,-0.65538
2,J011204.61-001442.4,0.044,17.9,51.2,14.836,1.93,225,2.78,39.09,39.40,8.3,SDSS J011204.61-001442.4,18.01925,-0.24512
3,J011515.78+001248.4,0.045,42.6,84.3,14.554,1.93,241,3.10,39.48,39.63,8.5,SDSS J011515.78+001248.4,18.81575,0.21346
4,J015127.10-083019.3,0.018,35.7,267.6,13.351,1.97,183,3.03,38.59,39.32,8.0,SDSS J015127.10-083019.3,27.86286,-8.50537
5,J020835.81-083754.8,0.034,28.4,186.8,13.694,1.93,242,2.97,39.06,39.73,8.5,SDSS J020835.81-083754.8,32.14917,-8.63197
6,J075354.98+130916.5,0.048,7.4,51.5,14.347,2.01,305,3.36,38.77,39.47,8.9,SDSS J075354.98+130916.5,118.47912,13.15475
7,J080716.58+145703.3,0.029,28.4,63.3,13.712,1.97,215,3.38,38.93,39.13,8.3,SDSS J080716.58+145703.3,121.81910,14.95092
8,J083158.49+562052.3,0.045,9.0,93.1,14.514,1.99,216,2.96,38.81,39.68,8.3,SDSS J083158.49+562052.3,127.99376,56.34786


In [20]:
first_fr0_xmatches = XMatch.query(
    cat1=fr0cat[0],
    cat2="vizier:VIII/92/first14",
    colRA1="_RA",
    colDec1="_DE",
    colRA2="RAJ2000",
    colDec2="DEJ2000",
    max_distance=10 * u.arcsec,
)

nvss_fr0_xmatches = XMatch.query(
    cat1=fr0cat[0],
    cat2="vizier:VIII/65/nvss",
    colRA1="_RA",
    colDec1="_DE",
    colRA2="RAJ2000",
    colDec2="DEJ2000",
    max_distance=5 * u.arcsec,
)

morx_fr0_xmatches = XMatch.query(
    cat1=fr0cat[0],
    cat2="vizier:V/158/morxv2",
    colRA1="_RA",
    colDec1="_DE",
    colRA2="RAJ2000",
    colDec2="DEJ2000",
    max_distance=1 * u.arcsec,
)

In [21]:
first_fr0_xmatches[0]

angDist,recno,SDSS,z,F1.4GHz,F[OIII],rmag,Dn,sigma*,Cr,logLr,logL[OIII],logMBH,SimbadName,_RA,_DE,_RAJ2000,_DEJ2000,FIRST,FITS,RAJ2000,DEJ2000,p(S),Fpeak,Fint,Rms,Maj,Min,PA,N1,c1,N2,Ep,Ep(JD),s_Ep(JD)
arcsec,,,,,,,,,,,,,,,,deg,deg,,,"""h:m:s""","""d:m:s""",,mJy,mJy,mJy,arcsec,arcsec,deg,,,,yr,d,d
float64,int16,str19,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,str24,float64,float64,float64,float64,str16,str4,str12,str12,float32,float64,float64,float32,float32,float32,float32,int16,str1,int16,float64,float64,float64
0.037064,1,J010852.48-003919.4,0.045,10.9,115.7,15.095,2.0,223,3.43,38.89,39.77,8.3,SDSS J010852.48-003919.4,17.21868,-0.65538,17.218675,-0.655389,J010852.4-003919,FITS,01 08 52.482,-00 39 19.40,0.014,12.87,13.27,0.137,1.43,0.0,88.5,1,g,1,1997.611,2450672.5,1075.642


In [22]:
fr0cat[0]

recno,SDSS,z,F1.4GHz,F[OIII],rmag,Dn,sigma*,Cr,logLr,logL[OIII],logMBH,SimbadName,_RA,_DE
,,,mJy,1e-20 W / m2,mag,,km / s,,log(1e-07W),log(1e-07W),log(solMass),,deg,deg
int32,str19,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,str24,float64,float64
1,J010852.48-003919.4,0.045,10.9,115.7,15.095,2.00,223,3.43,38.89,39.77,8.3,SDSS J010852.48-003919.4,17.21868,-0.65538
2,J011204.61-001442.4,0.044,17.9,51.2,14.836,1.93,225,2.78,39.09,39.40,8.3,SDSS J011204.61-001442.4,18.01925,-0.24512
3,J011515.78+001248.4,0.045,42.6,84.3,14.554,1.93,241,3.10,39.48,39.63,8.5,SDSS J011515.78+001248.4,18.81575,0.21346
4,J015127.10-083019.3,0.018,35.7,267.6,13.351,1.97,183,3.03,38.59,39.32,8.0,SDSS J015127.10-083019.3,27.86286,-8.50537
5,J020835.81-083754.8,0.034,28.4,186.8,13.694,1.93,242,2.97,39.06,39.73,8.5,SDSS J020835.81-083754.8,32.14917,-8.63197
6,J075354.98+130916.5,0.048,7.4,51.5,14.347,2.01,305,3.36,38.77,39.47,8.9,SDSS J075354.98+130916.5,118.47912,13.15475
7,J080716.58+145703.3,0.029,28.4,63.3,13.712,1.97,215,3.38,38.93,39.13,8.3,SDSS J080716.58+145703.3,121.81910,14.95092
8,J083158.49+562052.3,0.045,9.0,93.1,14.514,1.99,216,2.96,38.81,39.68,8.3,SDSS J083158.49+562052.3,127.99376,56.34786


In [23]:
# let us do the same for FR0 galaxies
# we do not need to XMatch with NVSS as there is already a NVSS flux measurement for the FR0 sources
# we will not search for FIRST counterparts as there is already the NVSS flux measurement at 1.4 GHz


for row in fr0cat[0]:
    sdss_id = row["SimbadName"]
    ngc_id = get_source_identifier(sdss_id, "NGC")
    first_id = get_source_identifier(sdss_id, "FIRST")
    ic_id = get_source_identifier(sdss_id, "IC")
    nvss_id = get_source_identifier(sdss_id, "NVSS")
    name = ngc_id if ngc_id else ic_id
    fermi_id = get_source_identifier(sdss_id, "4FGL")
    distance = Distance(z=row["z"]).to("Mpc")
    L_OIII_FR0 = row["logL[OIII]"]
    L_NVSS = np.power(10, row["logLr"])
   
    # FIRST measurement
    this_source_first_xmatch = first_fr0_xmatches["SimbadName"] == sdss_id
    if this_source_first_xmatch.any():
        first_xmatch_name = (
            "FIRST " + first_fr0_xmatches["FIRST"][this_source_first_xmatch][0]
        )
        first_xmatch_name_nospace = (
                "FIRST" + first_fr0_xmatches["FIRST"][this_source_first_xmatch][0]
            )
        first_xmatch_flux = first_fr0_xmatches["Fint"][this_source_first_xmatch][0]
        first_xmatch_flux_err = first_fr0_xmatches["Rms"][this_source_first_xmatch][0]
    else:
        first_xmatch_name = ""
        first_xmatch_flux = 0
        first_xmatch_name_nospace = ""
        first_xmatch_flux_err = 0
            
    first_morx_match = first_xmatch_name_nospace == morx_fr0_xmatches["FIRST-ID"]
    if first_morx_match.any():
        morx_first_name = morx_fr0_xmatches["FIRST-ID"][first_morx_match][0]
    else:
        morx_first_name = ""   

    # NVSS cross match with Nagar 2005 measurement
    this_source_nvss_xmatch = nvss_fr0_xmatches["SimbadName"] == sdss_id
    if this_source_nvss_xmatch.any():
        nvss_xmatch_name = (
            "NVSS J" + nvss_fr0_xmatches["NVSS"][this_source_nvss_xmatch][0]
        )
        nvss_xmatch_flux = nvss_fr0_xmatches["S1.4"][this_source_nvss_xmatch][0]
        nvss_xmatch_flux_err = nvss_fr0_xmatches["e_S1.4"][this_source_nvss_xmatch][0]
    else:
        nvss_xmatch_name = ""
        nvss_xmatch_flux = 0
        nvss_xmatch_flux_err = 0

        # NVSS Cross matched with Nagar and then with MORX 
    nvss_morx_match = nvss_xmatch_name == morx_fr0_xmatches["NVSS-ID"]
    if nvss_morx_match.any():
        morx_nvss_name = morx_fr0_xmatches["NVSS-ID"][nvss_morx_match][0]
    else:
        morx_nvss_name = ""
    
    morx_matches = morx_fr0_xmatches["SimbadName"] == sdss_id
    if morx_matches.any():
        morx_xmm = morx_fr0_xmatches["XMM-ID"][morx_matches][0]
        xmm4_name = morx_xmm.removeprefix("4XMM ")
        morx_cxo = morx_fr0_xmatches["CX-ID"][morx_matches][0]
        cx0_name = morx_cxo.removeprefix("CXOG ")
        morx_swift = morx_fr0_xmatches["Swift-ID"][morx_matches][0]
        swift_name = insert_space_source_ids(morx_swift)
        morx_first = morx_fr0_xmatches["FIRST-ID"][morx_matches][0]
        morx_nvss = morx_fr0_xmatches["NVSS-ID"][morx_matches][0]
        morx_lotss = morx_fr0_xmatches["LoTSS-ID"][morx_matches][0]
        morx_vlass = morx_fr0_xmatches["VLASS-ID"][morx_matches][0]
        morx_lobedist = morx_fr0_xmatches["Lobedist"][morx_matches][0]
    else:
        morx_xmm = ""
        morx_cxo = ""
        xmm4_name = ""
        cx0_name = ""
        morx_swift = ""
        swift_name = ""
        morx_first = ""
        morx_nvss = ""
        morx_lotss = ""
        morx_vlass = ""
        morx_lobedist = 0

    morx_4xmm_match = xmm4[0]["4XMM"] == xmm4_name
    if morx_4xmm_match.any():
        xmm_flux4 = xmm4[0]["Flux4"][morx_4xmm_match][0]
        xmm_flux4_err = xmm4[0]["e_Flux4"][morx_4xmm_match][0]
        xmm_flux5 = xmm4[0]["Flux5"][morx_4xmm_match][0]
        xmm_flux5_err = xmm4[0]["e_Flux5"][morx_4xmm_match][0]
        xmm_flux9 = xmm4[0]["Flux9"][morx_4xmm_match][0]
        xmm_flux9_err = xmm4[0]["e_Flux9"][morx_4xmm_match][0]
        HR = xmm4[0]["HR4"][morx_4xmm_match][0]
    else:
        xmm_flux4 = 0
        xmm_flux4_err = 0
        xmm_flux5 = 0
        xmm_flux5_err = 0
        xmm_flux9 = 0
        xmm_flux9_err = 0
        HR = 0

    morx_cxo_match = cxo[0]["CXO"] == cx0_name
    if morx_cxo_match.any():
        cxo_fb = cxo[0]["Fb"][morx_cxo_match][0]
        cxo_fb_uplim = cxo[0]["B_Fb"][morx_cxo_match][0]
        cxo_fb_lowlim = cxo[0]["b_Fb"][morx_cxo_match][0]
        cxo_fw= cxo[0]["Fw"][morx_cxo_match][0]
        cxo_fw_uplim = cxo[0]["B_Fw"][morx_cxo_match][0]
        cxo_fw_lowlim = cxo[0]["b_Fw"][morx_cxo_match][0]
    else:
        cxo_fb = 0
        cxo_fb_uplim = 0
        cxo_fb_lowlim = 0
        cxo_fw = 0
        cxo_fw_uplim = 0
        cxo_fw_lowlim = 0

    morx_swift_match = swift["IAUName"] == swift_name
    if morx_swift_match.any():
        swift_flux = swift["PowFlux"][morx_swift_match][0]
        swift_flux_pos = swift["PowFlux_pos"][morx_swift_match][0]
        swift_flux_neg = swift["PowFlux_neg"][morx_swift_match][0]
    else: 
        swift_flux = 0
        swift_flux_pos = 0
        swift_flux_neg = 0
        # check if the source is in the list of sources detected by Torresi et al. 2018
    torresi_detection = sdss_id in torresi_sources

    fr0_catalogue.add_row(
         [
            name,
            "FR0",
            sdss_id,
            fermi_id,
            nvss_id,
            nvss_xmatch_name,
            morx_nvss_name,
            first_id,
            first_xmatch_name,
            morx_first_name,
            morx_xmm,
            morx_cxo,
            morx_swift,
            morx_lotss,
            morx_vlass,
            morx_lobedist,
            distance.to_value("Mpc"),
            L_OIII_FR0,
            torresi_detection,
            convert_F_nu_to_luminosity(1.4 * u.GHz, nvss_xmatch_flux, u.mJy, distance),
            convert_F_nu_to_luminosity(1.4 * u.GHz, nvss_xmatch_flux_err, u.mJy, distance),
            convert_F_nu_to_luminosity(1.4 * u.GHz, first_xmatch_flux, u.mJy, distance),
            convert_F_nu_to_luminosity(1.4 * u.GHz, first_xmatch_flux_err, u.mJy, distance),
            convert_flux_to_luminosity(xmm_flux4,'mW / m**2', distance),
            convert_flux_to_luminosity(xmm_flux4_err,'mW / m**2', distance),
            convert_flux_to_luminosity(xmm_flux5,'mW / m**2', distance),
            convert_flux_to_luminosity(xmm_flux5_err,'mW / m**2', distance),
            convert_flux_to_luminosity(xmm_flux9,'mW / m**2', distance),
            convert_flux_to_luminosity(xmm_flux9_err,'mW / m**2', distance),
            convert_flux_to_luminosity(cxo_fb,'mW / m**2', distance),
            convert_flux_to_luminosity(cxo_fb_lowlim,'mW / m**2', distance),
            convert_flux_to_luminosity(cxo_fb_uplim,'mW / m**2', distance),
            convert_flux_to_luminosity(cxo_fw,'mW / m**2', distance),
            convert_flux_to_luminosity(cxo_fw_lowlim,'mW / m**2', distance),
            convert_flux_to_luminosity(cxo_fw_uplim,'mW / m**2', distance),
            convert_flux_to_luminosity(swift_flux,'erg / (cm**2)*(s)', distance),
            convert_flux_to_luminosity(swift_flux_neg,'erg / (cm**2)*(s)', distance),
            convert_flux_to_luminosity(swift_flux_pos,'erg / (cm**2)*(s)', distance),
            HR
        ])

09/15/2025 12:14:33 PM|utils|NGC counterpart not available for SDSS J010852.48-003919.4
09/15/2025 12:14:33 PM|utils|SDSS J010852.48-003919.4 matched with FIRST J010852.4-003919 by SIMBAD
09/15/2025 12:14:33 PM|utils|IC counterpart not available for SDSS J010852.48-003919.4
09/15/2025 12:14:33 PM|utils|SDSS J010852.48-003919.4 matched with NVSS J010852-003922 by SIMBAD
09/15/2025 12:14:33 PM|utils|4FGL counterpart not available for SDSS J010852.48-003919.4
/opt/anaconda3/lib/python3.12/site-packages/astropy/table/table.py:3220: UserWarning: Units from inserted quantities will be ignored.
  self.insert_row(len(self), vals, mask)
09/15/2025 12:14:33 PM|utils|NGC counterpart not available for SDSS J011204.61-001442.4
09/15/2025 12:14:33 PM|utils|SDSS J011204.61-001442.4 matched with FIRST J011204.6-001442 by SIMBAD
09/15/2025 12:14:33 PM|utils|IC counterpart not available for SDSS J011204.61-001442.4
09/15/2025 12:14:33 PM|utils|SDSS J011204.61-001442.4 matched with NVSS J011204-001441 by

In [34]:
fr0_catalogue

SOURCE_NAME,SOURCE_TYPE,SIMBAD SDSS-ID,SIMBAD FERMI-ID,SIMBAD NVSS-ID,NVSS-XMATCH-ID,NVSS-MORX-ID,SIMBAD FIRST-ID,FIRST-XMATCH-ID,FIRST-MORX-ID,XMM-MORX-ID,CXO-MORX-ID,SWIFT-MORX-ID,LoTSS-MORX-ID,VLASS-MORX,LOBE EXTENSION,DISTANCE,Log10(L_OIII),TORESSI DETECTION,NVSS-FLUX-XMATCH,NVSS-FLUX-ERROR-XMATCH,FIRST-FLUX-XMATCH,FIRST-FLUX-ERROR-XMATCH,4XMM Mean Flux (2.0-4.5keV band),4XMM Mean Flux Error (2.0-4.5keV band),4XMM Mean Flux (4.5-12keV band),4XMM Mean Flux Error (4.5-12keV band),4XMM Mean Flux (0.2-12keV band),4XMM Mean Flux Error (0.2-12keV band),ACIS Flux F_b,F_b lowlim,F_b uplim,HRC Flux F_w,F_w lowlim,F_w uplim,Swift PowerLaw-Flux,Swift Flux Neg,Swift Flux Pos,Hardness Ratio 4-5
,,,,,,,,,,,,,,,mas,Mpc,erg / s,,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,
str10,str3,str24,str17,str19,str19,str1,str22,str22,str21,str21,str21,str21,str22,str22,int16,float32,float32,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,FR0,SDSS J010852.48-003919.4,,NVSS J010852-003922,NVSS J010852-003922,,FIRST J010852.4-003919,FIRST J010852.4-003919,FIRSTJ010852.4-003919,,,,,VLAJ010852.48-003919.0,0,206.15471,39.77,False,7.617443993590242e+38,3.5595532680328238e+37,9.447054373359114e+38,9.753175564024665e+36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
,FR0,SDSS J011204.61-001442.4,,NVSS J011204-001441,NVSS J011204-001441,,FIRST J011204.6-001442,FIRST J011204.6-001442,FIRSTJ011204.6-001442,,,,,VLAJ011204.62-001442.1,0,201.42886,39.4,False,1.1961759114491608e+39,4.757517748607297e+37,1.2695776151062681e+39,7.204241199793214e+36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
,FR0,SDSS J011515.78+001248.4,,NVSS J011515+001247,NVSS J011515+001247,,FIRST J011515.7+001248,FIRST J011515.7+001248,FIRSTJ011515.7+001248,4XMM J011515.7+001248,CXOG J011515.7+001248,,,VLAJ011515.77+001248.7,0,206.15471,39.63,True,3.018501171291834e+39,9.254838157419886e+37,3.311808360577739e+39,7.759825937605555e+36,8.857541501411906e+40,6.508287195271214e+39,5.161097984842734e+40,1.4137731968450824e+40,4.176978122677506e+41,9.142458315137446e+39,2.3543902313097247e+41,2.196752959484764e+41,2.512027675423765e+41,0.0,0.0,0.0,0.0,0.0,0.0,-0.71128
NGC 707,FR0,SDSS J015127.10-083019.3,,NVSS J015127-083019,NVSS J015127-083019,,FIRST J015127.1-083019,FIRST J015127.1-083019,FIRSTJ015127.1-083019,,,,,VLAJ015127.11-083019.0,0,80.84892,39.32,False,4.007454241711921e+38,1.642399279390131e+37,3.539917913512196e+38,2.573092197851564e+36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
,FR0,SDSS J020835.81-083754.8,,NVSS J020835-083755,NVSS J020835-083755,,FIRST J020835.8-083755,FIRST J020835.8-083755,FIRSTJ020835.8-083755,,,,,VLAJ020835.82-083755.0,0,154.52747,39.73,False,1.1279750682804154e+39,3.5999203353168535e+37,1.0383770486723258e+39,5.759872441141641e+36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
,FR0,SDSS J075354.98+130916.5,,NVSS J075355+130915,NVSS J075355+130915,,FIRST J075354.9+130916,FIRST J075354.9+130916,FIRSTJ075354.9+130916,,,,,VLAJ075354.98+130916.6,0,220.37094,39.47,False,6.0197622384528654e+38,4.067406917873558e+37,1.026613506071286e+39,1.1307391115319044e+37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
,FR0,SDSS J080716.58+145703.3,,NVSS J080716+145703,NVSS J080716+145703,,FIRST J080716.5+145703,FIRST J080716.5+145703,FIRSTJ080716.5+145703,,,,,VLAJ080716.56+145703.5,0,131.32161,39.13,False,8.926266228431127e+38,3.7553869486481945e+37,7.3663362079286e+38,4.073150684850659e+36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
,FR0,SDSS J083158.49+562052.3,,NVSS J083158+562050,NVSS J083158+562050,,FIRST J083158.5+562052,FIRST J083158.5+562052,FIRSTJ083158.5+562052,,,,ILTJ083158.51+562052.4,VLAJ083158.43+562052.3,0,206.15471,39.68,False,6.3360048

In [36]:
coreG_catalogue['L_5Ghz'] = 3.57 * coreG_catalogue['NVSS-FLUX-XMATCH']
fr0_catalogue['L_5Ghz'] = 3.57 * fr0_catalogue['NVSS-FLUX-XMATCH']

In [26]:
np.array(coreG_catalogue["Swift PowerLaw-Flux"])

array([0.00000000e+00, 0.00000000e+00, 1.28444917e+41, 6.18539030e+41,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.75294680e+40, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 4.44062623e+40, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 3.23442537e+40, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 4.02874632e+40,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [27]:
coreG_catalogue[3]

SOURCE_NAME,SOURCE_TYPE,SIMBAD SDSS-ID,SIMBAD FERMI-ID,SIMBAD NVSS-ID,NVSS-XMATCH-ID,NVSS-MORX-ID,SIMBAD FIRST-ID,FIRST-XMATCH-ID,FIRST-MORX-ID,XMM-MORX-ID,CXO-MORX-ID,SWIFT-MORX-ID,LoTSS-MORX-ID,VLASS-MORX,LOBE EXTENSION,DISTANCE,Log10(L_OIII),TORESSI DETECTION,NVSS-FLUX-XMATCH,NVSS-FLUX-ERROR-XMATCH,FIRST-FLUX-XMATCH,FIRST-FLUX-ERROR-XMATCH,4XMM Mean Flux (2.0-4.5keV band),4XMM Mean Flux Error (2.0-4.5keV band),4XMM Mean Flux (4.5-12keV band),4XMM Mean Flux Error (4.5-12keV band),4XMM Mean Flux (0.2-12keV band),4XMM Mean Flux Error (0.2-12keV band),ACIS Flux F_b,F_b lowlim,F_b uplim,HRC Flux F_w,F_w lowlim,F_w uplim,Swift PowerLaw-Flux,Swift Flux Neg,Swift Flux Pos,Hardness Ratio 4-5
,,,,,,,,,,,,,,,mas,Mpc,erg / s,,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,
str7,str9,str24,str17,str19,str19,str1,str22,str22,str21,str21,str21,str21,str22,str22,int16,float32,float32,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
NGC315,L1.9,,4FGL J0057.7+3023,NVSS J005748+302114,NVSS J005748+302114,,,,,4XMM J005748.8+302109,CXO J005748.4+302108,LSXPSJ005748.9+302107,ILDJ005748.85+302109.2,VLAJ005748.74+302110.2,47,65.8,39.430813,False,5.599695253952314e+39,1.834895553072984e+38,0.0,0.0,1.4905547322603484e+41,2.739438909435452e+39,2.3181019336632667e+41,7.264777167249235e+39,4.374127740731979e+41,3.2466409138359184e+39,0.0,0.0,0.0,0.0,0.0,0.0,6.1853902975666635e+41,-1.966581126266547e+40,1.966581126266547e+40,-0.452047


In [37]:
path1 = Path("./")
path1.mkdir(exist_ok=True, parents=True)
fr0_catalogue.write(
    path1 / "fr0_catalogue.fits", overwrite=True
)

In [38]:
path2 = Path("./")
path2.mkdir(exist_ok=True, parents=True)
coreG_catalogue.write(
    path2 / "coreG_catalogue.fits", overwrite=True
)

In [39]:
from astropy.table import vstack

fr0_coreG_catalogue = vstack([fr0_catalogue,coreG_catalogue])

In [40]:
fr0_coreG_catalogue

SOURCE_NAME,SOURCE_TYPE,SIMBAD SDSS-ID,SIMBAD FERMI-ID,SIMBAD NVSS-ID,NVSS-XMATCH-ID,NVSS-MORX-ID,SIMBAD FIRST-ID,FIRST-XMATCH-ID,FIRST-MORX-ID,XMM-MORX-ID,CXO-MORX-ID,SWIFT-MORX-ID,LoTSS-MORX-ID,VLASS-MORX,LOBE EXTENSION,DISTANCE,Log10(L_OIII),TORESSI DETECTION,NVSS-FLUX-XMATCH,NVSS-FLUX-ERROR-XMATCH,FIRST-FLUX-XMATCH,FIRST-FLUX-ERROR-XMATCH,4XMM Mean Flux (2.0-4.5keV band),4XMM Mean Flux Error (2.0-4.5keV band),4XMM Mean Flux (4.5-12keV band),4XMM Mean Flux Error (4.5-12keV band),4XMM Mean Flux (0.2-12keV band),4XMM Mean Flux Error (0.2-12keV band),ACIS Flux F_b,F_b lowlim,F_b uplim,HRC Flux F_w,F_w lowlim,F_w uplim,Swift PowerLaw-Flux,Swift Flux Neg,Swift Flux Pos,Hardness Ratio 4-5,L_5Ghz
,,,,,,,,,,,,,,,mas,Mpc,erg / s,,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,,erg / s
str10,str9,str24,str17,str19,str19,str1,str22,str22,str21,str21,str21,str21,str22,str22,int16,float32,float32,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,FR0,SDSS J010852.48-003919.4,,NVSS J010852-003922,NVSS J010852-003922,,FIRST J010852.4-003919,FIRST J010852.4-003919,FIRSTJ010852.4-003919,,,,,VLAJ010852.48-003919.0,0,206.15471,39.77,False,7.617443993590242e+38,3.5595532680328238e+37,9.447054373359114e+38,9.753175564024665e+36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.719427505711716e+39
,FR0,SDSS J011204.61-001442.4,,NVSS J011204-001441,NVSS J011204-001441,,FIRST J011204.6-001442,FIRST J011204.6-001442,FIRSTJ011204.6-001442,,,,,VLAJ011204.62-001442.1,0,201.42886,39.4,False,1.1961759114491608e+39,4.757517748607297e+37,1.2695776151062681e+39,7.204241199793214e+36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.270348003873504e+39
,FR0,SDSS J011515.78+001248.4,,NVSS J011515+001247,NVSS J011515+001247,,FIRST J011515.7+001248,FIRST J011515.7+001248,FIRSTJ011515.7+001248,4XMM J011515.7+001248,CXOG J011515.7+001248,,,VLAJ011515.77+001248.7,0,206.15471,39.63,True,3.018501171291834e+39,9.254838157419886e+37,3.311808360577739e+39,7.759825937605555e+36,8.857541501411906e+40,6.508287195271214e+39,5.161097984842734e+40,1.4137731968450824e+40,4.176978122677506e+41,9.142458315137446e+39,2.3543902313097247e+41,2.196752959484764e+41,2.512027675423765e+41,0.0,0.0,0.0,0.0,0.0,0.0,-0.71128,1.0776049181511848e+40
NGC 707,FR0,SDSS J015127.10-083019.3,,NVSS J015127-083019,NVSS J015127-083019,,FIRST J015127.1-083019,FIRST J015127.1-083019,FIRSTJ015127.1-083019,,,,,VLAJ015127.11-083019.0,0,80.84892,39.32,False,4.007454241711921e+38,1.642399279390131e+37,3.539917913512196e+38,2.573092197851564e+36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.4306611642911558e+39
,FR0,SDSS J020835.81-083754.8,,NVSS J020835-083755,NVSS J020835-083755,,FIRST J020835.8-083755,FIRST J020835.8-083755,FIRSTJ020835.8-083755,,,,,VLAJ020835.82-083755.0,0,154.52747,39.73,False,1.1279750682804154e+39,3.5999203353168535e+37,1.0383770486723258e+39,5.759872441141641e+36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.026870993761083e+39
,FR0,SDSS J075354.98+130916.5,,NVSS J075355+130915,NVSS J075355+130915,,FIRST J075354.9+130916,FIRST J075354.9+130916,FIRSTJ075354.9+130916,,,,,VLAJ075354.98+130916.6,0,220.37094,39.47,False,6.0197622384528654e+38,4.067406917873558e+37,1.026613506071286e+39,1.1307391115319044e+37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1490551191276728e+39
,FR0,SDSS J080716.58+145703.3,,NVSS J080716+145703,NVSS J080716+145703,,FIRST J080716.5+145703,FIRST J080716.5+145703,FIRSTJ080716.5+145703,,,,,VLAJ080716.56+145703.5,0,131.32161,39.13,False,8.926266228431127e+38,3.7553869486481945e+37,7.3663362079286e+38,4.073150684850659e+36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.186677043549912e+39
,FR0,SDSS J083158.49+562052.3,,NVSS J08315

In [41]:
path = Path("./")
path.mkdir(exist_ok=True, parents=True)
fr0_coreG_catalogue.write(
    path / "fr0_coreG_catalogue.fits", overwrite=True
)

In [33]:
fr0cat[0]

recno,SDSS,z,F1.4GHz,F[OIII],rmag,Dn,sigma*,Cr,logLr,logL[OIII],logMBH,SimbadName,_RA,_DE
,,,mJy,1e-20 W / m2,mag,,km / s,,log(1e-07W),log(1e-07W),log(solMass),,deg,deg
int32,str19,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,str24,float64,float64
1,J010852.48-003919.4,0.045,10.9,115.7,15.095,2.00,223,3.43,38.89,39.77,8.3,SDSS J010852.48-003919.4,17.21868,-0.65538
2,J011204.61-001442.4,0.044,17.9,51.2,14.836,1.93,225,2.78,39.09,39.40,8.3,SDSS J011204.61-001442.4,18.01925,-0.24512
3,J011515.78+001248.4,0.045,42.6,84.3,14.554,1.93,241,3.10,39.48,39.63,8.5,SDSS J011515.78+001248.4,18.81575,0.21346
4,J015127.10-083019.3,0.018,35.7,267.6,13.351,1.97,183,3.03,38.59,39.32,8.0,SDSS J015127.10-083019.3,27.86286,-8.50537
5,J020835.81-083754.8,0.034,28.4,186.8,13.694,1.93,242,2.97,39.06,39.73,8.5,SDSS J020835.81-083754.8,32.14917,-8.63197
6,J075354.98+130916.5,0.048,7.4,51.5,14.347,2.01,305,3.36,38.77,39.47,8.9,SDSS J075354.98+130916.5,118.47912,13.15475
7,J080716.58+145703.3,0.029,28.4,63.3,13.712,1.97,215,3.38,38.93,39.13,8.3,SDSS J080716.58+145703.3,121.81910,14.95092
8,J083158.49+562052.3,0.045,9.0,93.1,14.514,1.99,216,2.96,38.81,39.68,8.3,SDSS J083158.49+562052.3,127.99376,56.34786
